# Doc2Vec model
> * Positive or Negative

In [4]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import word2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
mecab = Mecab()
ct = ctwitter()

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split

# Word2Vec 생성

* vector size
 [ 1000, 2000 ]

In [6]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None,encoding='utf-8')
print (rawdata.shape)

(491510, 2)


In [7]:
rawdata.head()

,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1


In [8]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [9]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

12

In [10]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [12]:
d2v_docs = list()
for idx in tqdm(rawdata.index):
    text = rawdata.loc[idx,0]
    pos = mecab.pos(text)
    pos = [x[0] for x in pos ]#if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    pos = [x for x in pos if not x in stopwords]
    label = rawdata.loc[idx,1]
    d2v_docs.append(TaggedDocument(pos, label))


  0%|          | 747/491510 [00:09<1:49:25, 74.75it/s]Exception in thread Thread-32:
Traceback (most recent call last):
  File "C:\Users\pc\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\pc\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\pc\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration
100%|██████████| 491510/491510 [1:01:09<00:00, 133.94it/s]


In [13]:
train, test = train_test_split(d2v_docs, test_size = 0.1)

In [ ]:
pickle.dump(d2v_docs,open('./data/pre_data/tagged_data/pre_data_d2v_docs_for_word2vec_sentiment_by_mecab.pickled','wb'))
pickle.dump(train,open('./data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled','wb'))
pickle.dump(test,open('./data/pre_data/train_test_Data/pre_data_test_for_word2vec_sentiment_by_mecab.pickled','wb'))

In [ ]:
x_train = [ x.words for x in tqdm(train)] 
y_train = [ x.tags for x in tqdm(train)] 
x_test = [ x.words for x in tqdm(test)] 
y_test = [ x.tags for x in tqdm(test)] 

100%|██████████| 49151/49151 [00:00<00:00, 1291507.09it/s]


In [ ]:
w2v_model = word2vec.Word2Vec(sg=1, size=3000,window=8, workers = 6, iter = 20)
w2v_model.build_vocab(tqdm(x_train))
w2v_model.train(tqdm(x_train), total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

100%|██████████| 442359/442359 [20:31<00:00, 359.16it/s]


In [ ]:
pickle.dump(w2v_model, open('./data/pre_data/pre_data_w2v_model_for_word2vec_sentiment_by_mecab.pickle','wb'))

In [ ]:
# importing bokeh library for interactive dataviz
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [w2v_model[w] for w in list(w2v_model.wv.vocab.keys())[:10000]]

# dimensionality reduction. converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)
# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(w2v_model.wv.vocab.keys())[:10000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform(x_train)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

In [ ]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, embeddings, merge
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=3000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, np.array(y_train), epochs=100, verbose=2)

In [ ]:
score = model.evaluate(test_vecs_w2v, np.array(y_test), verbose=2)
print (score[1])

0.835